In [1]:
# importing all libraries needed
import numpy as np
import pandas as pd
import os

In [2]:
# saving the main directory
main_directory = os.getcwd()
main_directory

'C:\\Users\\greci\\Documents\\Jupiter_notebooks\\forJuliana'

In [3]:
#Counting number of files on folder
list = os.listdir('files') 
number_files = len(list)
number_files

3

In [4]:
# Changing directory to the folder
os.chdir(r'C:\\Users\\greci\\Documents\\Jupiter_notebooks\\forJuliana\\files')
os.getcwd()


'C:\\Users\\greci\\Documents\\Jupiter_notebooks\\forJuliana\\files'

In [62]:
# Reading files
ThreeCT_data = pd.ExcelFile('Session1_07062021_Males_WT_PLex.xlsx')

# Reading sheets within files
df1 = pd.read_excel(ThreeCT_data, '1', header=None)
df2 = pd.read_excel(ThreeCT_data, '2', header=None)
df3 = pd.read_excel(ThreeCT_data, '3', header=None)
df4 = pd.read_excel(ThreeCT_data, '4', header=None)

#Dropping the frist 14 rows and the last two
rows_to_drop = np.arange(14)
rows_to_drop = np.append(rows_to_drop,[19,20])
df1 = df1.drop(df1.index[rows_to_drop])
df2 = df2.drop(df2.index[rows_to_drop])
df3 = df3.drop(df3.index[rows_to_drop])
df4 = df4.drop(df4.index[rows_to_drop])

#Creating and assigning new header
new_header = df1.iloc[0] #grab the first row for the header
df1 = df1[1:] #take the data less the header row
df1.columns = new_header #set the header row as the df header
df2 = df2[1:] #take the data less the header row
df2.columns = new_header #set the header row as the df header
df3 = df3[1:] #take the data less the header row
df3.columns = new_header #set the header row as the df header
df4 = df4[1:] #take the data less the header row
df4.columns = new_header #set the header row as the df header

# Selecting columns of av verlocity and visit latency for iso and empty
df1 = df1[["Avg. velocity (pixels/s)", "Visit latency (s)"]]
df1.columns=["AVG_VEL_PIXELS/s)", "VISIT_LATENCY_ISO_s", "VISIT_LATENCY_EMPTY_s"]
df1.reset_index(drop=True, inplace=True)

# Adding time in minutes' column
TIME_minutes = [5,10,15,20]# Creating time-column vector


df1

,AVG_VEL_PIXELS/s),VISIT_LATENCY_ISO_s,VISIT_LATENCY_EMPTY_s
0,40.2,76.2,44.8
1,27.2,35.1,7.2
2,28.7,124.2,53.5
3,25.4,14.9,85.8


In [19]:
# Saving group's name
file = 'Session1_07062021_Males_WT_PLex'
print(file)
group_name = file[0:8]
group_name

Session1_07062021_Males_WT_PLex


'Session1'

In [ ]:
xls = pd.ExcelFile('path_to_file.xls')






#Using a loop to process all files and populate the main data-frame

i = 0
while i < number_files:
    
    ##############################################
    # Processing each file
    ##############################################
    # Saving sample's number and reviewer's name
    file = list[i]
    sample = file[5:7]


    # Reading file's data
    regions_count_activity_fracc = pd.read_csv(file)
    regions_count_activity_fracc = regions_count_activity_fracc[regions_count_activity_fracc.PIXEL_COUNT != "PIXEL_COUNT"]

    # Adding-up regions' fracctions
    regions_fracc_1 = regions_count_activity_fracc[["INTENSITY_1", "INTENSITY_1_PERC"]]
    regions_fracc_2 = regions_count_activity_fracc[["INTENSITY_2", "INTENSITY_2_PERC"]]
    regions_fracc_3 = regions_count_activity_fracc[["INTENSITY_3", "INTENSITY_3_PERC"]]
    regions_fracc_1 = regions_fracc_1.rename(columns={"INTENSITY_1": "INTENSITY","INTENSITY_1_PERC" : "COUNTS"})
    regions_fracc_2 = regions_fracc_2.rename(columns={"INTENSITY_2": "INTENSITY","INTENSITY_2_PERC" : "COUNTS"})
    regions_fracc_3 = regions_fracc_3.rename(columns={"INTENSITY_3": "INTENSITY","INTENSITY_3_PERC" : "COUNTS"})
    total_region_activity = regions_fracc_1.append(regions_fracc_2, ignore_index=True).append(regions_fracc_3, ignore_index=True)
    total_region_activity["COUNTS"] = total_region_activity["COUNTS"].astype(float)
    total_region_activity = total_region_activity.groupby(['INTENSITY']).agg('sum').reset_index()
    total_region_activity["COUNTS"] = total_region_activity["COUNTS"].round(0)
    total_region_activity["INTENSITY"] = total_region_activity["INTENSITY"].astype(int)

    # Adding activity columns to intensitites template data-frame
    output = intensities_template.merge(total_region_activity, how='left', left_on='LabelID', right_on='INTENSITY')
    output = output.drop(columns=['INTENSITY'])
    activity_label_name = 'ACTIVITY'
    output = output.rename(columns = {'COUNTS':activity_label_name})

    # Adding a column of the sample number
    sample_column = np.empty(len(output))
    sample = int(sample)
    sample_column.fill(sample)
    output['SAMPLE'] = sample_column
    ##########################################################################################################################
    
    # store DataFrame in list
    appended_data.append(output)
    
    i += 1
    
# see pd.concat documentation for more info
appended_data = pd.concat(appended_data)
end = time.time()


print(i,' Files processed ')
print(' ')